----
## Exploring the Data

In [1]:
# Import libraries necessary for this project
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder,StandardScaler,OneHotEncoder,MinMaxScaler

In [2]:
# Load the data and print out a few lines and Performing operations to inspect data
df=pd.read_csv('transactions-2022-10-15.csv')
print('the shape of the dataset is {} rows and {} columns'.format(df.shape[0],df.shape[1]))
print('-'*127)
df.head(4)

the shape of the dataset is 132688 rows and 23 columns
-------------------------------------------------------------------------------------------------------------------------------


,Transaction Number,Transaction Date,Property ID,Transaction Type,Transaction sub type,Registration type,Is Free Hold?,Usage,Area,Property Type,...,Property Size (sq.m),Room(s),Parking,Nearest Metro,Nearest Mall,Nearest Landmark,No. of Buyer,No. of Seller,Master Project,Project
0,102-1-2019,2019-01-02 08:11:24,305887533,Sales,Sell - Pre registration,Off-Plan,Free Hold,Residential,JUMEIRAH VILLAGE CIRCLE,Unit,...,68.82,1 B/R,P3-106,Dubai Internet City,Mall of the Emirates,Sports City Swimming Academy,1,1,NaN,Bloom Towers
1,102-1-2020,2020-01-02 07:29:48,664106052,Sales,Sell - Pre registration,Off-Plan,Free Hold,Residential,Al Wasl,Unit,...,23.63,Studio,EU,Buj Khalifa Dubai Mall Metro Station,Dubai Mall,Burj Khalifa,1,1,NaN,Rove City Walk
2,102-1-2021,2021-01-03 07:10:43,859526640,Sales,Sell - Pre registration,Off-Plan,Free Hold,Residential,LA MER,Building,...,264.62,3 B/R,NaN,Trade Centre Metro Station,Dubai Mall,Burj Khalifa,1,1,NaN,Sur La Mer
3,102-10-2019,2019-01-02 11:17:19,126906900,Sales,Sell - Pre registration,Off-Plan,Free Hold,Residential,DUBAI SPORTS CITY,Unit,...,112.58,2 B/R,P1-022,Dubai Internet City,Marina Mall,Sports City Swimming Academy,2,1,NaN,Hera Tower


In [3]:
#Summary Information and data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132688 entries, 0 to 132687
Data columns (total 23 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Transaction Number       132688 non-null  object 
 1   Transaction Date         132688 non-null  object 
 2   Property ID              132688 non-null  int64  
 3   Transaction Type         132688 non-null  object 
 4   Transaction sub type     132688 non-null  object 
 5   Registration type        132688 non-null  object 
 6   Is Free Hold?            132688 non-null  object 
 7   Usage                    132688 non-null  object 
 8   Area                     132688 non-null  object 
 9   Property Type            132688 non-null  object 
 10  Property Sub Type        130456 non-null  object 
 11  Amount                   132688 non-null  float64
 12  Transaction Size (sq.m)  132618 non-null  float64
 13  Property Size (sq.m)     132688 non-null  float64
 14  Room

which indicates that there is so many null values the need to be filled or dropped

In [4]:
#convert Transaction Date column from object to DateTime for useability on analysis
df['Transaction Date']=pd.to_datetime(df['Transaction Date'])

In [5]:
# a function that takes the dataset and return a dataframe with number of unique values for each columns 
def unique_df(dfs):
    uni=[]
    for i in list(df.columns):
        uni.append(i)
        uni.append(df[i].nunique())
    dct = {uni[i]: uni[i + 1] for i in range(0, len(uni), 2)}
    df1=pd.DataFrame(dct.items(),columns=['coulmn name','no of unique values']).sort_values('no of unique values',ascending=False)
    return df1

In [6]:
#calling the function sorted in descending order 
unique_df(df)

,coulmn name,no of unique values
0,Transaction Number,132636
1,Transaction Date,132164
2,Property ID,124597
11,Amount,38058
12,Transaction Size (sq.m),31419
13,Property Size (sq.m),31184
15,Parking,25707
22,Project,1221
8,Area,254
16,Nearest Metro,56


the result indicates that master project and Transaction Type has only 1 unique values so there is so much need on them but also checking for nulls in this columns is important to determine to drop them 

In [7]:
#checking accurate numbers of nulls in every column
df.isna().sum().sort_values(ascending=False)

Master Project             132502
Parking                     43158
Nearest Mall                37945
Nearest Metro               37637
Project                     24026
Room(s)                     20137
Nearest Landmark            17137
Property Sub Type            2232
Transaction Size (sq.m)        70
No. of Seller                   0
No. of Buyer                    0
Property Size (sq.m)            0
Transaction Number              0
Transaction Date                0
Property Type                   0
Area                            0
Usage                           0
Is Free Hold?                   0
Registration type               0
Transaction sub type            0
Transaction Type                0
Property ID                     0
Amount                          0
dtype: int64

### Handling null values

In [8]:
df[['Transaction Size (sq.m)','Property Size (sq.m)']].corr()

,Transaction Size (sq.m),Property Size (sq.m)
Transaction Size (sq.m),1.000000,0.988243
Property Size (sq.m),0.988243,1.000000


In [9]:
Size = (df['Transaction Size (sq.m)'] - df['Property Size (sq.m)']).mean()
df['Transaction Size (sq.m)'].fillna(df['Property Size (sq.m)'] + Size, inplace=True)

for Transaction Size coulmn there are low null values but we can fill it with Property Size column because of the correlated between them

In [10]:
# Creating a function to impute null values
def impute(*cols):
    imp = SimpleImputer(strategy='constant')
    for i in list(cols):
        df[i]=imp.fit_transform(df[i].values.reshape(-1,1))

In [11]:
impute('Property Sub Type','Nearest Landmark','Room(s)','Project','Nearest Metro','Nearest Mall','Parking')

for Property Sub Type column is category column so better than dropping it or filling it with the mode that will affect the distribution significantly its better to use SimpleImputer and that goes also to the rest of other columns

In [12]:
df.drop('Master Project',axis=1,inplace=True)

99.8% of Master Project data is nulls and it has only one unique value so it's not much significant so the best obtion is to drop it

In [13]:
#checking if all nulls have been cleaned
df.isna().sum().sort_values(ascending=False)

Transaction Number         0
Transaction Date           0
No. of Seller              0
No. of Buyer               0
Nearest Landmark           0
Nearest Mall               0
Nearest Metro              0
Parking                    0
Room(s)                    0
Property Size (sq.m)       0
Transaction Size (sq.m)    0
Amount                     0
Property Sub Type          0
Property Type              0
Area                       0
Usage                      0
Is Free Hold?              0
Registration type          0
Transaction sub type       0
Transaction Type           0
Property ID                0
Project                    0
dtype: int64

In [14]:
df['Transaction Date_year'] = df['Transaction Date'].dt.year
df['Transaction Date_month'] = df['Transaction Date'].dt.month
df['Transaction Date_week'] = df['Transaction Date'].dt.week
df['Transaction Date_day'] = df['Transaction Date'].dt.day
df['Transaction Date_dayofweek'] = df['Transaction Date'].dt.dayofweek

In [15]:
#sorting records by Date 
df=df.sort_values('Transaction Date').reset_index(drop=True)
df.head(5)

,Transaction Number,Transaction Date,Property ID,Transaction Type,Transaction sub type,Registration type,Is Free Hold?,Usage,Area,Property Type,...,Nearest Mall,Nearest Landmark,No. of Buyer,No. of Seller,Project,Transaction Date_year,Transaction Date_month,Transaction Date_week,Transaction Date_day,Transaction Date_dayofweek
0,11-1-2019,2019-01-02 06:59:05,470790696,Sales,Sale,Ready,Free Hold,Residential,PALM JUMEIRAH,Unit,...,Mall of the Emirates,Burj Al Arab,1,1,ROYAL AMWAJ,2019,1,1,2,2
1,11-2-2019,2019-01-02 06:59:28,470775186,Sales,Sale,Ready,Free Hold,Residential,PALM JUMEIRAH,Unit,...,Mall of the Emirates,Burj Al Arab,1,1,ROYAL AMWAJ,2019,1,1,2,2
2,11-3-2019,2019-01-02 06:59:42,470773494,Sales,Sale,Ready,Free Hold,Residential,PALM JUMEIRAH,Unit,...,Mall of the Emirates,Burj Al Arab,1,1,ROYAL AMWAJ,2019,1,1,2,2
3,11-4-2019,2019-01-02 06:59:57,470791824,Sales,Sale,Ready,Free Hold,Residential,PALM JUMEIRAH,Unit,...,Mall of the Emirates,Burj Al Arab,1,1,ROYAL AMWAJ,2019,1,1,2,2
4,11-5-2019,2019-01-02 07:00:12,26352975,Sales,Sale,Ready,Free Hold,Residential,PALM JUMEIRAH,Unit,...,Mall of the Emirates,Burj Al Arab,1,1,ROYAL AMWAJ,2019,1,1,2,2


In [16]:
df.to_csv('transactions-2022-10-15(modified).csv')